In [1]:
try:
    from google.colab import drive
except ModuleNotFoundError:
    import os
    os.chdir("FireBallPipe")
else:
    drive.mount("/content/gdrive", force_remount=True)
    ! cd /content/gdrive/MyDrive/FIREBall-2/FB2_2023/instrument_alignment_focusing/XY_calibration/FireBallPipe

# cd /content/gdrive/MyDrive/FIREBall-2/FB2_2023/instrument_alignment_focusing/XY_calibration
# ! git clone https://github.com/vpicouet/FireBallPipe.git

In [7]:
# %matplotlib widget
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
from astropy.io import fits
from astropy import wcs, coordinates
from astropy import units as u
from astropy.wcs.utils import proj_plane_pixel_scales
from matplotlib import pyplot as plt
from astropy.table import Table, vstack 
import matplotlib.patches as patches
import unittest
from mpl_interactions import heatmap_slicer
import pandas as pd
import datetime 
# os.chdir(os.path.dirname(os.path.realpath(__file__)))
sys.path.insert(1, './Calibration')
from guider2UV.guider2UV import Guider2UV, diff_skycoord, fit_model, plot_fit
from guider2UV.MaskAstrometry import LocalScienceMaskProjector
from Calibration.mapping import Mapping

# encoder gains 2022:
Elg = 0.9969 # 2023 #1.0090 2022 # Elg = 1.00379 # 2018
CEg = 1.018  # 2023 #1.0187 2022 # CEg = 1.02928 # 2018
print("After calibration of the encoders, El is %0.5f ''/inc and Ce is %0.5f ''/inc" %(Elg*0.092,CEg*0.01))





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
After calibration of the encoders, El is 0.09171 ''/inc and Ce is 0.01018 ''/inc


In [15]:
# prerequisites:
Field = "F3"
previous_xy_mapping = "Calibration/Mappings/2023/XY/F3_XY_230620.pkl"
mask_det_map_file = "Calibration/Mappings/2023/mask_to_det_mapping/mapping-mask-det-w-2023-0-F3_59.5.pkl"
target_filename = 'Calibration/Targets/2022/targets_%s.csv'%(Field)
G2UV = Guider2UV(filename=previous_xy_mapping)
mask_det_map = Mapping(mask_det_map_file)
Slits = Table.read(target_filename, format='ascii')
print(G2UV)




FOV center angular position in guider <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (250.42402654, 36.43499699)>): (lon, lat) in deg
    (0.18232866, -0.00944478)>
FOV center pixel position in guider [array(1357.21182339), array(504.44777814)]

Guider2UV object:
    Local Field Projector: 
ScienceMaskProjector object:
    LocalFrame: <SkyOffsetICRS Frame (rotation=-90.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (352.3424, 0.21245)>)>
    gamma: 1.006815157811007

    Guider Field Projector: 
GuiderMaskProjector object:
    LocalFrame: <SkyOffsetICRS Frame (rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (250.42402654, 36.43499699)>)>
    wcs: WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 250.424026537  36.4349969891  
CRPIX : 640.0  540.0  
PC1_1 PC1_2  : 1.0  0.0  
PC2_1 PC2_2  : 0.0  1.0  
CDELT : 0.00024648264068189  0.00024664057706659  
NAXIS : 0  0

    mask_rotation: 38.37

In [22]:
# slit_pos = np.array([st[st['Internal-count']==s][x][0], st[st['Internal-count']==s][y][0]])

G2UV.SienceMask2guider(np.array(Slits["RA"][0],Slits["DEC"][0]), world=True, angle=False)

AttributeError: 'numpy.ndarray' object has no attribute 'transform_to'

In [17]:
from pprint import pprint
pprint(vars(G2UV))

{'FOV_center_guider_coord': <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (250.42402654, 36.43499699)>): (lon, lat) in deg
    (0.18232866, -0.00944478)>,
 'FieldP': <guider2UV.MaskAstrometry.LocalScienceMaskProjector object at 0x7faff8be5f10>,
 'GuiderP': <guider2UV.MaskAstrometry.LocalGuiderMaskProjector object at 0x7fafd896dd90>,
 'filename': 'Calibration/Mappings/2023/XY/F3_XY_230620.pkl',
 'mask_rotation': <Quantity 38.37087573 arcmin>}
